# 这是dip第一次作业


## requirements 
```
numpy
gradio==3.36.1   
python-opencv
```




### 以下为import部分

In [2]:
import gradio as gr
import cv2
import numpy as np

## global transform

### 老师给的代码


In [2]:
import gradio as gr
import cv2
import numpy as np

# Function to convert 2x3 affine matrix to 3x3 for matrix multiplication

def to_3x3(affine_matrix):
    return np.vstack([affine_matrix, [0, 0, 1]])

# Function to apply transformations based on user inputs
def apply_transform(image, scale, rotation, translation_x, translation_y, flip_horizontal):

    # Convert the image from PIL format to a NumPy array
    image = np.array(image)
    # Pad the image to avoid boundary issues
    pad_size = min(image.shape[0], image.shape[1]) // 2
    image_new = np.zeros((pad_size*2+image.shape[0], pad_size*2+image.shape[1], 3), dtype=np.uint8) + np.array((255,255,255), dtype=np.uint8).reshape(1,1,3)
    image_new[pad_size:pad_size+image.shape[0], pad_size:pad_size+image.shape[1]] = image
    image = np.array(image_new)
    transformed_image = np.array(image)

    ### todo : FILL: Apply Composition Transform
    # Note: for scale and rotation, implement them around the center of the image （围绕图像中心进行放缩和旋转）

    return transformed_image

# Gradio Interface
def interactive_transform():
    with gr.Blocks() as demo:
        gr.Markdown("## Image Transformation Playground")
        
        # Define the layout
        with gr.Row():
            # Left: Image input and sliders
            with gr.Column():
                image_input = gr.Image(type="pil", label="Upload Image")

                scale = gr.Slider(minimum=0.1, maximum=2.0, step=0.1, value=1.0, label="Scale")
                rotation = gr.Slider(minimum=-180, maximum=180, step=1, value=0, label="Rotation (degrees)")
                translation_x = gr.Slider(minimum=-300, maximum=300, step=10, value=0, label="Translation X")
                translation_y = gr.Slider(minimum=-300, maximum=300, step=10, value=0, label="Translation Y")
                flip_horizontal = gr.Checkbox(label="Flip Horizontal")
            
            # Right: Output image
            image_output = gr.Image(label="Transformed Image")
        
        # Automatically update the output when any slider or checkbox is changed
        inputs = [
            image_input, scale, rotation, 
            translation_x, translation_y, 
            flip_horizontal
        ]

        # Link inputs to the transformation function
        image_input.change(apply_transform, inputs, image_output)
        scale.change(apply_transform, inputs, image_output)
        rotation.change(apply_transform, inputs, image_output)
        translation_x.change(apply_transform, inputs, image_output)
        translation_y.change(apply_transform, inputs, image_output)
        flip_horizontal.change(apply_transform, inputs, image_output)

    return demo

# Launch the Gradio interface
interactive_transform().launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


### 补充后的代码



- 计算出图像的中心点
- 然后基于该中心点构建缩放、旋转和平移的仿射变换矩阵
- 接着，我们将这些变换组合起来应用到图像上



#### 1. 计算图像中心点
图像中心点可以通过图像的尺寸来计算：
```python
center_x, center_y = image.shape[1] // 2, image.shape[0] // 2
```

#### 2. 构建缩放矩阵
缩放矩阵的基本形式是：
\[
\text{Scale Matrix} = \begin{bmatrix} s & 0 & 0 \\ 0 & s & 0 \\ 0 & 0 & 1 \end{bmatrix}
\]
其中 `s` 是缩放因子。

#### 4. 构建平移矩阵
平移矩阵的基本形式是：
\[
\text{Translation Matrix} = \begin{bmatrix} 1 & 0 & t_x \\ 0 & 1 & t_y \\ 0 & 0 & 1 \end{bmatrix}
\]
其中 `t_x` 和 `t_y` 是 X 和 Y 方向上的平移量。

#### 5. 组合所有变换矩阵
将这些矩阵组合在一起，并围绕图像中心进行变换。

#### 最终代码实现：





In [ ]:
def apply_transform(image, scale, rotation, translation_x, translation_y, flip_horizontal):
    # Convert the image from PIL format to a NumPy array
    image = np.array(image)
    
    # Pad the image to avoid boundary issues
    pad_size = min(image.shape[0], image.shape[1]) // 2
    image_new = np.zeros((pad_size*2 + image.shape[0], pad_size*2 + image.shape[1], 3), dtype=np.uint8) + np.array([255, 255, 255], dtype=np.uint8).reshape(1, 1, 3)
    image_new[pad_size:pad_size+image.shape[0], pad_size:pad_size+image.shape[1]] = image
    image = np.array(image_new)

    # Get image dimensions
    h, w = image.shape[:2]
    
    # Calculate the center of the image
    center_x, center_y = w // 2, h // 2
    
    # Create the scale matrix
    scale_matrix = np.array([[scale, 0, 0], [0, scale, 0]])

    # Create the rotation matrix (around the center)
    theta = np.radians(rotation)
    cos_theta, sin_theta = np.cos(theta), np.sin(theta)
    rotation_matrix = np.array([[cos_theta, -sin_theta, 0], [sin_theta, cos_theta, 0]])

    # Create the translation matrix
    translation_matrix = np.array([[1, 0, translation_x], [0, 1, translation_y]])

    # Convert to 3x3 matrix by appending [0, 0, 1]
    scale_matrix_3x3 = to_3x3(scale_matrix)
    rotation_matrix_3x3 = to_3x3(rotation_matrix)
    translation_matrix_3x3 = to_3x3(translation_matrix)

    # Combine transformations: T = Translation * Rotation * Scale
    transform_matrix = translation_matrix_3x3 @ rotation_matrix_3x3 @ scale_matrix_3x3

    # Translate the image center to origin for rotation/scale
    transform_to_origin = np.array([[1, 0, -center_x], [0, 1, -center_y], [0, 0, 1]])
    transform_back = np.array([[1, 0, center_x], [0, 1, center_y], [0, 0, 1]])

    # Final transformation matrix
    final_transform = transform_back @ transform_matrix @ transform_to_origin

    # Apply the affine transformation using OpenCV
    transformed_image = cv2.warpAffine(image, final_transform[:2], (w, h), borderValue=(255, 255, 255))

    # Apply horizontal flip if required
    if flip_horizontal:
        transformed_image = cv2.flip(transformed_image, 1)

    return transformed_image

## point transform


In [ ]:
## global transformation


### 老师给的代码

In [ ]:
import cv2
import numpy as np
import gradio as gr

# todo 初始化全局变量，存储控制点和目标点
points_src = []
points_dst = []
image = None


# todo 上传图像时清空控制点和目标点
def upload_image(img):
    global image, points_src, points_dst
    points_src.clear()  # 清空控制点
    points_dst.clear()  # 清空目标点
    image = img
    return img

# todo 记录点击点事件，并标记点在图像上，同时在成对的点间画箭头
def record_points(evt: gr.SelectData):
    global points_src, points_dst, image
    x, y = evt.index[0], evt.index[1]  # 获取点击的坐标
    
    # 判断奇偶次来分别记录控制点和目标点
    if len(points_src) == len(points_dst):
        points_src.append([x, y])  # 奇数次点击为控制点
    else:
        points_dst.append([x, y])  # 偶数次点击为目标点
    
    # 在图像上标记点（蓝色：控制点，红色：目标点），并画箭头
    marked_image = image.copy()
    for pt in points_src:
        cv2.circle(marked_image, tuple(pt), 1, (255, 0, 0), -1)  # 蓝色表示控制点
    for pt in points_dst:
        cv2.circle(marked_image, tuple(pt), 1, (0, 0, 255), -1)  # 红色表示目标点
    
    # 画出箭头，表示从控制点到目标点的映射
    for i in range(min(len(points_src), len(points_dst))):
        cv2.arrowedLine(marked_image, tuple(points_src[i]), tuple(points_dst[i]), (0, 255, 0), 1)  # 绿色箭头表示映射
    
    return marked_image

# todo 执行仿射变换

def point_guided_deformation(image, source_pts, target_pts, alpha=1.0, eps=1e-8):
    """ 
    Return
    ------
        A deformed image.
    """
    
    warped_image = np.array(image)
    ### ! todo: FILL: 基于MLS or RBF 实现 image warping
    

    return warped_image






def run_warping():
    global points_src, points_dst, image ### fetch global variables

    warped_image = point_guided_deformation(image, np.array(points_src), np.array(points_dst))

    return warped_image

# 清除选中点
def clear_points():
    global points_src, points_dst
    points_src.clear()
    points_dst.clear()
    return image  # 返回未标记的原图

# 使用 Gradio 构建界面
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            input_image = gr.Image(source="upload", label="上传图片", interactive=True, width=800, height=200)
            point_select = gr.Image(label="点击选择控制点和目标点", interactive=True, width=800, height=800)
            
        with gr.Column():
            result_image = gr.Image(label="变换结果", width=800, height=400)
    
    # 按钮
    run_button = gr.Button("Run Warping")
    clear_button = gr.Button("Clear Points")  # 添加清除按钮
    
    # 上传图像的交互
    input_image.upload(upload_image, input_image, point_select)
    # 选择点的交互，点选后刷新图像
    point_select.select(record_points, None, point_select)
    # 点击运行 warping 按钮，计算并显示变换后的图像
    run_button.click(run_warping, None, result_image)
    # 点击清除按钮，清空所有已选择的点
    clear_button.click(clear_points, None, point_select)
    
# 启动 Gradio 应用
demo.launch()


### 补充后的代码

In [ ]:
import cv2
import numpy as np
import gradio as gr
from scipy.interpolate import Rbf

# 初始化全局变量，存储控制点和目标点
points_src = []
points_dst = []
image = None


# 上传图像时清空控制点和目标点
def upload_image(img):
    global image, points_src, points_dst
    points_src.clear()  # 清空控制点
    points_dst.clear()  # 清空目标点
    image = img
    return img

# 记录点击点事件，并标记点在图像上，同时在成对的点间画箭头
'''
evt.index 提供了用户点击的坐标 (x, y)。
偶数次点击作为源点（蓝色），奇数次点击作为目标点（红色）。
在图像上用圆圈标记点击的点，并在成对的点之间画出绿色箭头，表示映射关系。
最终返回标记了点的图像供用户查看。
'''
def record_points(evt: gr.SelectData):
    global points_src, points_dst, image
    x, y = evt.index[0], evt.index[1]  # 获取点击的坐标
    
    # 判断奇偶次来分别记录控制点和目标点
    if len(points_src) == len(points_dst):
        points_src.append([x, y])  # 奇数次点击为控制点
    else:
        points_dst.append([x, y])  # 偶数次点击为目标点
    
    # 在图像上标记点（蓝色：控制点，红色：目标点），并画箭头
    marked_image = image.copy()
    for pt in points_src:
        cv2.circle(marked_image, tuple(pt), 1, (255, 0, 0), -1)  # 蓝色表示控制点
    for pt in points_dst:
        cv2.circle(marked_image, tuple(pt), 1, (0, 0, 255), -1)  # 红色表示目标点
    
    # 画出箭头，表示从控制点到目标点的映射
    for i in range(min(len(points_src), len(points_dst))):
        cv2.arrowedLine(marked_image, tuple(points_src[i]), tuple(points_dst[i]), (0, 255, 0), 1)  # 绿色箭头表示映射
    
    return marked_image






# 执行仿射变换

def point_guided_deformation(image, source_pts, target_pts, alpha=1.0, eps=1e-8):

    #todo: FILL: 基于MLS or RBF 实现 image warping
    h, w, c = image.shape
    warped_image = np.zeros_like(image)


    # 如果没有足够的控制点，直接返回原图
    if len(source_pts) < 3:
        return image

    # 提取源点的 x, y 坐标
    src_x, src_y = source_pts[:, 0], source_pts[:, 1]
    dst_x, dst_y = target_pts[:, 0], target_pts[:, 1]

    # 基于 RBF 插值构建映射关系
    rbf_x = Rbf(src_x, src_y, dst_x, function=func, epsilon=epsilon)
    rbf_y = Rbf(src_x, src_y, dst_y, function=func, epsilon=epsilon)

    # 创建网格，包含图像中的所有像素点
    grid_x, grid_y = np.meshgrid(np.arange(w), np.arange(h))
    grid_x_flat = grid_x.flatten()
    grid_y_flat = grid_y.flatten()

    # 通过 RBF 计算每个像素的新位置
    new_x = rbf_x(grid_x_flat, grid_y_flat).reshape(h, w)
    new_y = rbf_y(grid_x_flat, grid_y_flat).reshape(h, w)

    # 将新的坐标点限制在图像的边界内
    new_x = np.clip(new_x, 0, w - 1).astype(np.float32)
    new_y = np.clip(new_y, 0, h - 1).astype(np.float32)

    # 使用 OpenCV 的 remap 函数进行插值，将图像映射到新的坐标
    warped_image = cv2.remap(image, new_x, new_y, cv2.INTER_LINEAR)

    return warped_image






# def run_warping():
#     global points_src, points_dst, image # fetch global variables
#
#     warped_image = point_guided_deformation(image, np.array(points_src), np.array(points_dst))
#
#     return warped_image

def run_warping():
    global points_src, points_dst, image

    # 确保点集的数量相等且不为空
    if len(points_src) == len(points_dst) and len(points_src) > 0:
        warped_image = point_guided_deformation(image, np.array(points_src), np.array(points_dst))
        return warped_image
    else:
        return image  # 如果点不匹配，则返回原图

# 清除选中点
def clear_points():
    global points_src, points_dst
    points_src.clear()
    points_dst.clear()
    return image  # 返回未标记的原图

# 使用 Gradio 构建界面
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            input_image = gr.Image(source="upload", label="上传图片", interactive=True, width=800, height=200)
            point_select = gr.Image(label="点击选择控制点和目标点", interactive=True, width=800, height=800)
            
        with gr.Column():
            result_image = gr.Image(label="变换结果", width=800, height=400)
    
    # 按钮
    run_button = gr.Button("Run Warping")
    clear_button = gr.Button("Clear Points")  # 添加清除按钮
    
    # 上传图像的交互
    input_image.upload(upload_image, input_image, point_select)
    # 选择点的交互，点选后刷新图像
    point_select.select(record_points, None, point_select)
    # 点击运行 warping 按钮，计算并显示变换后的图像
    run_button.click(run_warping, None, result_image)
    # 点击清除按钮，清空所有已选择的点
    clear_button.click(clear_points, None, point_select)
    
# 启动 Gradio 应用
demo.launch()
